<div style="
    background-color: #f4f4f4;
    border-left: 6px solid #007BFF;
    padding: 15px 20px;
    margin: 50px 0;
    border-radius: 8px;
    font-family: 'Arial', century gothic;
    color: #333;
">
    <h1 style="font-size: 28px; color: #007BFF; margin-top: 0;">
            English to French Machine Translation
    </h1>
</div>

## seq2seq LSTM

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000

In [2]:
# Read .txt data
data_path = r"C:\Users\ayman\Documents\AMIT_AI_DIPLOMA\Kaggle\fra-eng\fra.txt"


## Vectorization

In [3]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')       # get first and sec part of each line

    # \t => start sequence
    # \n => end sequence

    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)

    # Add missin chars
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)

    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)


In [4]:
# Sort characters
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_enc_tokens = len(input_characters)
num_dec_tokens = len(target_characters)
max_enc_seq_len = max([len(txt) for txt in input_texts])
max_dec_seq_len = max([len(txt) for txt in target_texts])

In [5]:
# Give indeces to the chars
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

## one-hot Encoder

In [6]:
encoder_input_data = np.zeros(
    (len(input_texts), max_enc_seq_len, num_enc_tokens), dtype='float32')

decoder_input_data = np.zeros(
    (len(input_texts), max_dec_seq_len, num_dec_tokens), dtype='float32')

decoded_target_data = np.zeros(
    (len(input_texts), max_dec_seq_len, num_dec_tokens), dtype='float32')

In [7]:
# one-hot encoding
# for encoder and decoder
# at seq(i), time-step(t) mark the char with 1
# then pad all the remaining from next-step(t+1) to max-len

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):

    # process the input_texts for the encoder
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1

    # process the target_texts for the decoder
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1

        if t > 0:
            decoded_target_data[i, t - 1, target_token_index[char]] = 1

    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1
    decoded_target_data[i, t:, target_token_index[' ']] = 1

In [8]:
# Encoder
encoder_inputs = Input(shape=(None, num_enc_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [9]:
# Decoder
decoder_inputs = Input(shape=(None, num_dec_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_dec_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [10]:
# define and compile model

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


In [11]:
# Train model
model.fit([encoder_input_data, decoder_input_data], decoded_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 38s 278ms/step - accuracy: 0.7056 - loss: 1.5519 - val_accuracy: 0.7113 - val_loss: 1.0606
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 41s 328ms/step - accuracy: 0.7454 - loss: 0.9596 - val_accuracy: 0.7264 - val_loss: 0.9597
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 28s 228ms/step - accuracy: 0.7627 - loss: 0.8608 - val_accuracy: 0.7535 - val_loss: 0.8732
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 84ms/step - accuracy: 0.7855 - loss: 0.7674 - val_accuracy: 0.7777 - val_loss: 0.7848
Epoch 5/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 83ms/step - accuracy: 0.8032 - loss: 0.6877 - val_accuracy: 0.7904 - val_loss: 0.7191
Epoch 6/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 93ms/step - accuracy: 0.8169 - loss: 0.6348 - val_accuracy: 0.7993 - val_loss: 0.6916
Epoch 7/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 97ms/step - accuracy: 0.8232 - loss: 0.6045 - val_accuracy: 0.8085 - val_loss: 0.6531
Epoch 8/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 103ms/step - accuracy: 0.8298 - los

## Model testing

In [14]:
# Define the encoder model for inference
encoder_model = Model(encoder_inputs, encoder_states)

# Define the decoder model for inference
# Inputs:
# - Current decoder input
# - Hidden state and cell state from the previous step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Pass inputs and previous states to the decoder LSTM
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]

# Pass decoder outputs through the dense layer to generate probabilities
decoder_outputs = decoder_dense(decoder_outputs)

# Define the decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)


In [17]:
# Define the maximum sequence length for the decoder
max_dec_seq_length = max([len(txt) for txt in target_texts])

# Reverse mapping of target token index to characters
reverse_target_char_index = {index: char for char, index in target_token_index.items()}


# Function to decode sequences
def decode_sequence(input_seq):
    # Encode the input sequence to get the initial states
    states_value = encoder_model.predict(input_seq)

    # Generate an empty target sequence of length 1 with the start token
    target_seq = np.zeros((1, 1, num_dec_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Initialize an empty string for the decoded sentence
    decoded_sentence = ""

    # Run the loop to generate the sequence
    while True:
        # Predict the next token using the decoder
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Get the token with the highest probability
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Break if we reach the end token or the maximum sequence length
        if sampled_char == "\n" or len(decoded_sentence) > max_dec_seq_length:
            break

        # Update the target sequence with the predicted token
        target_seq = np.zeros((1, 1, num_dec_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states for the next step
        states_value = [h, c]

    return decoded_sentence



In [18]:
# Testing the model
for seq_index in range(10):  # Test on the first 10 sequences
    # Prepare the input sequence
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    # Generate the decoded sentence
    decoded_sentence = decode_sequence(input_seq)
    # Print the results
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
-
Input sentence: Go.
Decoded sentence: Sarsez !

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
-
Input sentence: Go.
Decoded sentence: Sarsez !

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0